# Genetic Algorithm

This notebook showcases how to use the built-in Genetic Algorithm (GA)

In [1]:
using Statistics
using EvoLP
using OrderedCollections

For this example we will use the **Rosenbrock** function:

In [2]:
@doc rosenbrock

```
rosenbrock(x; a=1, b=5)
```

**Rosenbrock** benchmark function. With $a=1$ and $b=5$, minimum is at $f([a, a^2]) = 0$

$$
f(x) = (a - x_1)^2 + b(x_2 - x_1^2)^2
$$


In a GA, we use vectors as _individuals_.

Let's start creating the population. For that, we can use a generator. Let's use the normal generator:

In [3]:
@doc rand_pop_normal

```
rand_pop_normal(n, μ, Σ)
```

Generate a population of `n` vector individuals using a normal distribution with means `μ` and covariance `Σ`.

`μ` expects a vector of length *l* (i.e. length of an individual) while `Σ` expects an *l x l* matrix of covariances


In [4]:
pop_size = 50
population = rand_pop_normal(pop_size, [0, 0], [1 0; 0 1])
first(population, 3)

3-element Vector{Vector{Float64}}:
 [1.9508235863205539, 0.06400112093452046]
 [-0.4771474992721217, 1.6597904116187647]
 [-0.2619635641463636, -0.02039804264693687]

In a GA, we have _selection_, _crossover_ and _mutation_.

We can easily set up these operators using the built-ins provided by EvoLP.

Let's use rank based selection and single point crossover:

In [5]:
S = RankBasedSelectionGenerational()
C = SinglePointCrossover()

SinglePointCrossover()

For mutation, we can use a Gaussian approach:

In [6]:
@doc GaussianMutation

Gaussian mutation with standard deviation `σ`.


In [7]:
M = GaussianMutation(0.65)

GaussianMutation(0.65)

We can use the `Logbook` to record statistics about our run:

In [8]:
statnames = ["mean_eval", "max_f", "min_f", "median_f"]
fns = [mean, maximum, minimum, median]
thedict = LittleDict(statnames, fns)
thelogger = Logbook(thedict)

Logbook(LittleDict{AbstractString, Function, Vector{AbstractString}, Vector{Function}}("mean_eval" => Statistics.mean, "max_f" => maximum, "min_f" => minimum, "median_f" => Statistics.median), NamedTuple{(:mean_eval, :max_f, :min_f, :median_f)}[])

And now we're ready to use the `GA` built-in algorithm:

In [9]:
@doc GA

```
GA(f::Function, pop, k_max, S, C, M)
GA(logbook::Logbook, f::Function, pop, k_max, S, C, M)
```

Generational Genetic Algorithm.

## Arguments

  * `f`: Objective function to minimise
  * `pop`: Population—a list of individuals.
  * `k_max`: maximum iterations
  * `S::SelectionMethod`: a selection method. See selection.
  * `C::CrossoverMethod`: a crossover method. See crossover.
  * `M::MutationMethod`: a mutation method. See mutation.

Returns a `Result` type of the form:

$$
\big( f(x^*), x^*, pop, k_{max}, f_{calls} \big)
$$


In [10]:
result = GA(thelogger, rosenbrock, population, 200, S, C, M);

The output was suppressed so that we can analyse each part of the result separately:

In [11]:
@show optimum(result)

@show optimizer(result)

@show f_calls(result)

@show thelogger.records[end]

optimum(result) = 0.009004347905246778
optimizer(result) = [1.0176751433433628, 0.9939687236823952]


f_calls(result) = 20050
thelogger.records[end] = (mean_eval = 24.994362694190258, max_f = 353.97613737583487, min_f = 0.009004347905246778, median_f = 5.53320307967787)


(mean_eval = 24.994362694190258, max_f = 353.97613737583487, min_f = 0.009004347905246778, median_f = 5.53320307967787)